In [94]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

In [95]:
wine = datasets.load_wine()
X = wine.data
y = wine.target

In [96]:
print(wine.DESCR)

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

:Number of Instances: 178
:Number of Attributes: 13 numeric, predictive attributes and the class
:Attribute Information:
    - Alcohol
    - Malic acid
    - Ash
    - Alcalinity of ash
    - Magnesium
    - Total phenols
    - Flavanoids
    - Nonflavanoid phenols
    - Proanthocyanins
    - Color intensity
    - Hue
    - OD280/OD315 of diluted wines
    - Proline
    - class:
        - class_0
        - class_1
        - class_2

:Summary Statistics:

============================= ==== ===== ======= =====
                                Min   Max   Mean     SD
============================= ==== ===== ======= =====
Alcohol:                      11.0  14.8    13.0   0.8
Malic Acid:                   0.74  5.80    2.34  1.12
Ash:                          1.36  3.23    2.36  0.27
Alcalinity of Ash:            10.6  30.0    19.5   3.3
Magnesium:                    70.0 162.0    99.7  14.3

In [97]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=24)

In [98]:
baseline_svm = SVC()
baseline_svm.fit(X_train, y_train)
y_pred = baseline_svm.predict(X_test)

In [99]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Baseline SVM Accuracy: {accuracy:.2f}")

Baseline SVM Accuracy: 0.78


# importing gridsearchcv

In [100]:
from sklearn.model_selection import GridSearchCV

# defining parameter grid 

In [101]:
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': [0.1, 1, 'scale', 'auto']
}

# creating gridsearch object

In [102]:
grid_search = GridSearchCV(estimator=baseline_svm, param_grid=param_grid, cv=5)

# fitting model with grid of hyperparameters

In [103]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [0.1, 1, 10], 'gamma': [0.1, 1, 'scale', 'auto'],
                         'kernel': ['linear', 'rbf', 'poly']})

In [104]:
best_parame = grid_search.best_params_
best_model = grid_search.best_estimator_


In [105]:
y_predict_best = best_model.predict(X_test)
accuracy_best = accuracy_score(y_test, y_predict_best)
print(f"Best SVM Accuracy: {accuracy_best:.2f}")
print(f"Best Hyperparameters: {best_parame}")

Best SVM Accuracy: 0.94
Best Hyperparameters: {'C': 0.1, 'gamma': 0.1, 'kernel': 'poly'}


# RandomizedSearchCV

In [106]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

param_dist = {
    'C': uniform(0.1, 10), 
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto'] + list(np.logspace(-3, 3, 50))
}

In [107]:
randomized_search = RandomizedSearchCV(estimator=baseline_svm, param_distributions=param_dist, n_iter=20, cv=5)
randomized_search.fit(X_train, y_train)

RandomizedSearchCV(cv=5, estimator=SVC(), n_iter=20,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000028D64A03AA0>,
                                        'gamma': ['scale', 'auto', 0.001,
                                                  0.0013257113655901094,
                                                  0.0017575106248547913,
                                                  0.002329951810515372,
                                                  0.0030888435964774815,
                                                  0.004094915062380427,
                                                  0.005428675439323859,
                                                  0.0071968567300115215,
                                                  0.009540954...
                                                  0.022229964825261943,
                                                  0.029470517025518096,
                                                  0.03906939937054617,
                                                  0.0517947467923121,
                                                  0.06866488450043001,
                                                  0.09102981779915217,
                                                  0.1206792640639329,
                                                  0.15998587196060574,
                                                  0.21209508879201905,
                                                  0.28117686979742307,
                                                  0.3727593720314938,
                                                  0.49417133613238334,
                                                  0.655128556859551,
                                                  0.868511373751352,
                                                  1.151395399326447,
                                                  1.5264179671752334,
                                                  2.023589647725158, ...],
                                        'kernel': ['linear', 'rbf', 'poly']})

In [108]:
best_param = randomized_search.best_params_
best_model = randomized_search.best_estimator_

In [109]:
best_param

{'C': 0.8344801873969597, 'gamma': 2.6826957952797246, 'kernel': 'poly'}

In [110]:
best_model

SVC(C=0.8344801873969597, gamma=2.6826957952797246, kernel='poly')

In [111]:
y_pred_best = best_model.predict(X_test)
accuracy_best= accuracy_score(y_test, y_pred_best)
print(f"Best SVM Accuracy: {accuracy_best:.2f}")
print(f"Best Hyperparameters: {best_param}")

Best SVM Accuracy: 0.94
Best Hyperparameters: {'C': 0.8344801873969597, 'gamma': 2.6826957952797246, 'kernel': 'poly'}


# BayesSearchCV

In [112]:
from skopt import BayesSearchCV

In [113]:
param_space = {
    'C': (1e-6, 1e+6, 'log-uniform'),
    'gamma': (1e-6, 1e+1, 'log-uniform'),
    'kernel': ['linear', 'rbf']
}

In [114]:
bayes_search = BayesSearchCV(estimator=svm,search_spaces=param_space, n_iter=32,cv=5,random_state=42)
bayes_search.fit(X_train, y_train)

BayesSearchCV(cv=5, estimator=SVC(), n_iter=32, random_state=42,
              search_spaces={'C': (1e-06, 1000000.0, 'log-uniform'),
                             'gamma': (1e-06, 10.0, 'log-uniform'),
                             'kernel': ['linear', 'rbf']})

In [115]:
best_svm = bayes_search.best_estimator_
best_param = bayes_search.best_params_
print("Best SVM model:", best_svm)
print("Best Hyperparameters:", best_param)

Best SVM model: SVC(C=11185.625288472094, gamma=1.5247791391944723, kernel='linear')
Best Hyperparameters: OrderedDict({'C': 11185.625288472094, 'gamma': 1.5247791391944723, 'kernel': 'linear'})
